숙명여대 학과정보 수집

* 모듈 입력

In [ ]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

* 수집할 주소

In [ ]:
url = "https://www.sookmyung.ac.kr/sookmyungkr/1030/subview.do"

In [ ]:
# 접속객체 생성
session = requests.Session()

# 접속객체에 부가정보 삽입
session.headers.update({
    "Referer" : "",
    "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})

# 생성한 접속 객체를 활용해 API 접속
r = session.get(url)

# 접속 실패시 에러 메시지 생성
if r.status_code != 200:
    msg = "[%d Error] %s 에러 발생" %(r.status_code, r.reason)
    raise Exception(msg)

r.encoding = "utf-8"
# url 텍스트 추출
# print(r.text)
soup = BeautifulSoup(r.text)
soup

* 응답결과로부터 데이터 추출하기

In [ ]:
# 가독성을 위한 방법 1 : 구분 선 print("-"*10)
# 가독성을 위한 방법 2 : 쉼표를 넣어서 구분 print(a, )
# 가독성을 위한 방법 2 : end 파라미터를 사용 print(a, end="\n-------\n")

In [ ]:
# 수집한 정보를 저장할 리스트 생성
학과목록_리스트 = []

# 전체 웹 페이지에서 '학과'만 추출
college_list = soup.select(".college_list")
# print(college_list)

# item은 학과의 정보
for item in college_list:
    # print(item)
    # print("-"*30)


    # 학과 이름
    h5El = item.select("h5")
    name = h5El[0].text.strip()
    # print(name)
    # print("-"*30)


    # 상세보기 UR
    viewEl = item.select(".view")
    # href 속성을 가져오기 위해 attrs 내부의 href 호출
    view = viewEl[0].attrs['href']
    if view.find("https://www.sookmyung.ac.kr") == -1:
        view = "https://www.sookmyung.ac.kr" + view
    # print(view)
    # print("-"*30)
    

    #---------------------------------------
    # PDF 파일 크롤링, 저장을 위한 변수 변경(step3)
    #---------------------------------------
    # 학과소개 pdf
    pdfEl = item.select(".info")
    pdf = pdfEl[0].attrs['href']
    # print(pdf)
    # print("-"*30)

    # 학과홈페이지
    # soup은 웹 페이지 전체
    # item은 
    homepageEl = item.select(".homepage")
    # href 속성을 가져오기 위해 attrs 내부의 href 호출
    homepage = homepageEl[0].attrs['href']

    # 수집한 값들을 딕셔너리로 묶은 후 리스트에 추가
    college_dict = {"학과이름": name , "상세페이지": view, "홈페이지": homepage}
    

    #---------------------------------------
    # 상세보기 URL을 새롭게 수집03-학과수집(step2)
    # 기본 class 이후 class가 부재하므로 일일이 노가다해서 확인, 추출
    #---------------------------------------
    r = session.get(view)

    # 에러코드와 에러메시지 출력
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생" % (r.status_code, r.reason)
        print(msg)
        continue

    r.encoding = "utf-8"
    detailSoup = BeautifulSoup(r.text)
    # print(detailSoup)
    

    # 같은 구조를 갖는 전화번호, 팩스번호, 위치, 이메일주소 추출
    # 자손연산자를 사용 (자식 연산자는 '>'로 파일 하나하나 다 설정)
    infoEl = detailSoup.select(".college_info_data dl")
    # print(infoEl)
    # print("-" * 50)


    for info in infoEl:
        # print(info)
        # dt와 dd는 class 내부의 정보를 담고 있는 부분
        dt = info.select("dt")[0].text.replace(":", "").strip()
        dd = info.select("dd")[0].text.replace(":", "").strip()
        # print(dt, dd)
        # print("-" * 50)
        college_dict[dt] = dd
    #---------------------------------------
    # 상세보기 URL을 새롭게 수집03-학과수집(step2) 끝
    # 기본 class 이후 class가 부재하므로 일일이 노가다해서 확인, 추출
    #---------------------------------------
    학과목록_리스트.append(college_dict)
    # print(school_lsit)

    #---------------------------------------
    # PDF 파일 크롤링, 저장(step3)
    #---------------------------------------
    # pdf 파일 다운로드
    # url 저장하고 있는 변수는 pdf로 변경
    r = session.get(pdf, stream=True)
    # 정상 접근일 경우
    if r.status_code == 200:
        r.encoding = 'utf-8'
        with open("%s.pdf" % name, 'wb') as f:
            f.write(r.raw.read())


In [ ]:
# 상세보기 URL에 view 처리를 하는 이유
# 가져온 페이지 주소의 예) /sookmyungkr/1012/subview.do
# 같은 사이트 안에서 이동할 때는 앞부분 "https:.sookmyung.ac.kr"을 생략 가능하지만
# 외부에서 직접 접근할 때는 반드시 전체 주소가 필요함.
# 가져온 값이 전체주소가 아니므로 검사하여 누락된 부분을 덧붙여야 한다.
# 결론
# 내부에서 웹 사이트 접근 시 부분 주소가 가능하지만
# 외부에서 웹 사이트 접근 시 전체 주소가 요구됨

In [ ]:
df = DataFrame(학과목록_리스트)
df.to_excel("result3.xlsx", index = False)
df

In [ ]:
df = DataFrame(학과목록_리스트)
df.to_excel("result2.xlsx", index = False)
df